In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import nltk
import random

# Assicurati di avere i token di NLTK per il tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ANGELAANGUILANO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:

# Caricamento del testo (usa un libro dell'autore che desideri)
with open('testo_autore.txt', 'r', encoding='utf-8') as file:
    text = file.read().lower()

# Tokenizzazione del testo
tokens = nltk.word_tokenize(text)

# Creazione del vocabolario unico (mappa le parole in indici)
vocab = sorted(set(tokens))
vocab_size = len(vocab)
word_to_int = {word: i for i, word in enumerate(vocab)}
int_to_word = {i: word for i, word in enumerate(vocab)}

# Creazione delle sequenze di input (X) e target (y)
seq_length = 40  # Numero di parole in ciascuna sequenza
sequences = []

for i in range(seq_length, len(tokens)):
    seq = tokens[i-seq_length:i]  # Sequenza di 40 parole
    label = tokens[i]  # Parola successiva
    sequences.append([word_to_int[word] for word in seq] + [word_to_int[label]])

# Creazione degli input (X) e degli output (y)
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]


In [3]:

# Creazione del modello LSTM
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=seq_length-1))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Preparazione di y (one-hot encoding)
y = to_categorical(y, num_classes=vocab_size)

# Addestramento del modello
model.fit(X, y, epochs=50, batch_size=128)


Epoch 1/50


C:\Users\ANGELAANGUILANO\simple-environment\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 541ms/step - accuracy: 0.0365 - loss: 5.9359
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 542ms/step - accuracy: 0.0447 - loss: 5.3284
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 539ms/step - accuracy: 0.0529 - loss: 5.2133
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 512ms/step - accuracy: 0.0455 - loss: 5.1867
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 482ms/step - accuracy: 0.0484 - loss: 5.1744
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 492ms/step - accuracy: 0.0484 - loss: 5.1718
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 496ms/step - accuracy: 0.0425 - loss: 5.1698
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 481ms/step - accuracy: 0.0492 - loss: 5.1676
Epoch 9/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 505ms/step - accuracy: 0.0581 - loss: 5.1614
Epoch 10/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 442ms/step - accuracy: 0.0499 - loss: 5.1582
Epoch 11/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 471ms/step - accuracy: 0.0626 - loss: 5.1508
Epoch 12/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 513ms/step - acc

In [4]:

# Funzione per generare del testo
def generate_text(model, int_to_word, word_to_int, seed_text, seq_length, num_words):
    # Inizializza il testo con il seed_text
    result = seed_text
    input_text = seed_text.lower().split()
    
    # Genera 'num_words' parole
    for _ in range(num_words):
        # Converte il testo in sequenza numerica
        encoded = [word_to_int[word] for word in input_text if word in word_to_int]
        
        # Padding per la sequenza (poiché la sequenza di input deve avere una lunghezza fissa)
        encoded = pad_sequences([encoded], maxlen=seq_length-1, padding='pre')
        
        # Predice la parola successiva
        pred_probs = model.predict(encoded, verbose=0)
        predicted_idx = np.argmax(pred_probs)
        
        # Converte l'indice predetto in parola
        predicted_word = int_to_word[predicted_idx]
        
        # Aggiungi la parola predetta al testo
        result += ' ' + predicted_word
        
        # Aggiungi la parola predetta alla sequenza di input per la prossima previsione
        input_text.append(predicted_word)
        input_text = input_text[1:]
    
    return result


In [5]:
# Genera un testo nello stile dell'autore
seed_text = "era una notte buia"  # Testo iniziale
generated_text = generate_text(model, int_to_word, word_to_int, seed_text, seq_length, 100)
print(generated_text)

era una notte buia con la tuo partner con te . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo capoufficio . è il tuo
